In [0]:
-- Training Specialization Analysis: Finding Each Reindeer's Strength
WITH training_stats AS (
  SELECT 
    r.reindeer_name,
    t.training_type,
    AVG(t.speed_mph) as avg_speed,
    AVG(t.energy_efficiency) as avg_efficiency,
    COUNT(*) as training_sessions
  FROM fact_telemetry t
  JOIN dim_reindeer r ON t.reindeer_id = r.reindeer_id
  WHERE t.flight_status = 'FLYING'
    AND t.training_type IS NOT NULL
  GROUP BY r.reindeer_name, t.training_type
),
ranked_training AS (
  SELECT 
    reindeer_name,
    training_type,
    avg_speed,
    avg_efficiency,
    training_sessions,
    ROW_NUMBER() OVER (PARTITION BY reindeer_name ORDER BY avg_speed * avg_efficiency DESC) as rank
  FROM training_stats
  WHERE training_sessions >= 3  -- Minimum sessions for valid comparison
)
SELECT 
  reindeer_name,
  training_type as specialty,
  ROUND(avg_speed, 1) as speed_mph,
  ROUND(avg_efficiency * 100, 1) as efficiency_percent,
  training_sessions as sessions_completed,
  CASE training_type
    WHEN 'speed' THEN '💨 Speed Demon'
    WHEN 'endurance' THEN '🏃 Marathon Runner'
    WHEN 'navigation' THEN '🧭 Navigator'
    WHEN 'weather' THEN '🌪️ Storm Chaser'
    WHEN 'formation' THEN '👥 Team Player'
    ELSE '🎯 All-Rounder'
  END as specialty_badge
FROM ranked_training
WHERE rank = 1
ORDER BY speed_mph * avg_efficiency DESC;